# Quantization of DWT-ransformed audio signals

In [ ]:
import sounddevice as sd
import pywt
import math
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from scipy import signal
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import pylab

## Capture an audio sequence

In [ ]:
def plot(x, y, xlabel='', ylabel='', title=''):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_title(title)
    ax.grid()
    ax.xaxis.set_label_text(xlabel)
    ax.yaxis.set_label_text(ylabel)
    ax.plot(x, y, '.', markersize=1)
    plt.show(block=False)

In [ ]:
fs = 44100
duration = 80000/44100  # seconds
signal = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype=np.int16)
print("Say something!")
while sd.wait():
    pass
print("done")
signal = signal.flatten()

In [ ]:
plot(np.linspace(0, len(signal)-1, num=len(signal)), signal, "sample", "amplitude", "original")


## Select the number of levels of the DWT

In [ ]:
levels = 3

## Select a kernel filter

In [ ]:
#wavelet_name = "haar"
wavelet_name = "db5"
#wavelet_name = "db20"
#wavelet_name = "bior2.2"
#wavelet_name = "rbio2.2"
wavelet = pywt.Wavelet(wavelet_name)

## Let's compute basis fuctions of the inverse DWT
Inverse transform of a unit impulse. The frequency response is also shown.

In [ ]:
def deadzone_quantizer(x, quantization_step):
    k = (x / quantization_step).astype(np.int)
    return k

def deadzone_dequantizer(k, quantization_step):
    y = quantization_step * k
    return y

In [ ]:
chunk_size = 128
chunk_number = 15
#chunk = signal[chunk_size*chunk_number:chunk_size*(chunk_number+1)]
chunk_left = signal[chunk_size*(chunk_number-1):chunk_size*chunk_number]
chunk_center = signal[chunk_size*chunk_number:chunk_size*(chunk_number+1)]
chunk_right = signal[chunk_size*(chunk_number+1):chunk_size*(chunk_number+2)]
chunks = np.concatenate([chunk_left, chunk_center, chunk_right])

In [ ]:
def plot(y, xlabel='', ylabel='', title=''):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_title(title)
    ax.grid()
    ax.xaxis.set_label_text(xlabel)
    ax.yaxis.set_label_text(ylabel)
    x = np.linspace(0, len(y)-1, num=len(y))
    ax.plot(x, y, '-', markersize=1)
    plt.show(block=False)
#plot(signal[chunk_size*(chunk_number-1):chunk_size*(chunk_number+1)], "sample", "amplitude", "3 consecutive chunks")
plot(chunks, "sample", "amplitude", "3 consecutive chunks")

In [ ]:
quantization_step = 128
def transform_and_quantize(chunk):
    decomposition = pywt.wavedec(chunk, wavelet=wavelet, level=levels, mode="per")
    coefficients, slices = pywt.coeffs_to_array(decomposition)
    quantized_coeffs = deadzone_dequantizer(deadzone_quantizer(coefficients, quantization_step), quantization_step)
    decomposition = pywt.array_to_coeffs(quantized_coeffs, slices, output_format="wavedec")
    reconstructed_chunk = pywt.waverec(decomposition, wavelet=wavelet, mode="per")
    return reconstructed_chunk

rchunk_left = transform_and_quantize(chunk_left)
rchunk_center = transform_and_quantize(chunk_center)
rchunk_right = transform_and_quantize(chunk_right)
rchunks = np.concatenate([rchunk_left, rchunk_center, rchunk_right])

In [ ]:
plot(rchunks, "sample", "amplitude", "reconstructed chunks")

In [ ]:
ideal_chunks = transform_and_quantize(chunks)
plot(ideal_chunks, "sample", "amplitude", "reconstructed chunks")

## A solution: use the neighbor samples

In [ ]:
number_of_overlaped_samples = 1 << math.ceil(math.log(wavelet.dec_len * levels) / math.log(2))
print("number_of_overlaped_samples =", number_of_overlaped_samples)

In [ ]:
last_samples_left_chunk = chunk_left[chunk_size - number_of_overlaped_samples :]
first_samples_right_chunk = chunk_right[: number_of_overlaped_samples]
extended_chunk = np.concatenate([last_samples_left_chunk, chunk_center, first_samples_right_chunk])
print(len(last_samples_left_chunk), len(chunk_center), len(first_samples_right_chunk), len(extended_chunk))
plot(extended_chunk, "sample", "amplitude", "extended chunk")

In [ ]:
rextended_chunk = transform_and_quantize(extended_chunk)

In [ ]:
plot(rextended_chunk, "sample", "amplitude", "reconstructed extended chunk")

In [ ]:
rchunk = rextended_chunk[number_of_overlaped_samples : chunk_size + number_of_overlaped_samples]

In [ ]:
plot(rchunk, "sample", "amplitude", "reconstructed chunk with overlaping")

In [ ]:
plot(rchunk_center, "sample", "amplitude", "reconstructed chunk without overlaping")